In [ ]:
!pip -q install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import pycocotools.mask as mask_util
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from fastcore.all import *

In [ ]:
dataDir=Path('../input/sartorius-cell-instance-segmentation/')
register_coco_instances('all',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_all.json', dataDir)
all_metadata = MetadataCatalog.get('all')
all_dataset = DatasetCatalog.get('all')

In [ ]:
cache = {d['image_id']: d for d in all_dataset}

def vis(*image_ids, rect=None):
    l = len(image_ids)
    fig = plt.figure(figsize=(40, 12 * l))
    if rect:
        x, y, w, h = rect
    for num, image_id in enumerate(image_ids):
        d = cache[image_id]
        name = d["file_name"]
        img = cv2.imread(name)
        visualizer = Visualizer(img[:, :, ::-1], metadata=all_metadata, scale=1,instance_mode=ColorMode.SEGMENTATION)
        for a in d['annotations']:
            a['bbox'] = [-10,-10,0,0]
        out = visualizer.draw_dataset_dict(d)
        img2 = out.get_image()[:, :, ::-1]
        
        ax = fig.add_subplot(1 * l, 2, 2*num + 1)
        if rect:
            ax.add_patch(patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'))
        plt.imshow(img2)
        plt.title(d['image_id'])

        ax = fig.add_subplot(1 * l, 2, 2*num + 2)
        if rect:
            ax.add_patch(patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'))
        plt.imshow(img)

# Inconsistent annotations

In [ ]:
vis('35fc12883459', 'eec79772cb99', 'e92c56871769')

# Atefacts example

In [ ]:
vis('25559c20c6f3', '71d6e508abdc', rect=(300, 380, 80, 80))